# `TohuNamespaceNEW`


[TOC]


The `TohuNamespaceNEW` class allows grouping together other tohu generators and treating them as a single unit (which is used to implement the functionality of custom generators).

In [1]:
from tohu.tohu_namespace_NEW import TohuNamespaceNEW

## Initialisation and adding generators

In [2]:
from tohu import Integer, HashDigest, FakerGenerator

In [3]:
tohu_namespace = TohuNamespaceNEW()

tohu_namespace.add_generator("xx", Integer(100, 200))
tohu_namespace.add_generator("yy", HashDigest(length=6))
tohu_namespace.add_generator("zz", FakerGenerator(method="name"))

tohu_namespace.generators

{'xx': <Integer (id=c5bff4)>,
 'yy': <HashDigest (id=1518ac)>,
 'zz': <FakerGenerator (id=66af96)>}

The convenience method `add_tohu_generators_from_dict()` allows passing a dictionary, and it will call `add_generator()` for any tohu generators found in this dictionary (while ignoring any other values).

In [4]:
generators = {
    "xx": Integer(100, 200),
    "yy": HashDigest(length=6),
    "aa": "this will not be added because it is not a tohu generator",
    "zz": FakerGenerator(method="name"),
    "bb": 42
}

In [5]:
tohu_namespace = TohuNamespaceNEW()
tohu_namespace.add_tohu_generators_from_dict(generators)

In [6]:
tohu_namespace.generators

{'xx': <Integer (id=a45b46)>,
 'yy': <HashDigest (id=6661d1)>,
 'zz': <FakerGenerator (id=3430ab)>}

## Setting the `tohu_items_cls` attribute

Initially the `tohu_items_cls` attribute refers to a non-existent tohu items class:

In [7]:
tohu_namespace.tohu_items_cls

<NonExistentTohuItemsClass>

Once all desired generators have been added to the tohu namespace, we can call `set_tohu_items_class`, which will automatically create a tohu items class with the same field names as the generators contained in the namespace.

In [8]:
tohu_namespace.set_tohu_items_class(name="Quux")

In [9]:
tohu_namespace.tohu_items_cls

tohu.tohu_items_class.Quux

This items class can then be used to create individual tohu items.

In [10]:
tohu_namespace.tohu_items_cls(xx=100, yy="910A97", zz="Kristen Wallace")

Quux(xx=100, yy='910A97', zz='Kristen Wallace')

## Resetting and generating tohu items

In [11]:
tohu_namespace.generators

{'xx': <Integer (id=a45b46)>,
 'yy': <HashDigest (id=6661d1)>,
 'zz': <FakerGenerator (id=3430ab)>}

In [12]:
tohu_namespace.reset(seed=11111)

print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(xx=163, yy='7551AA', zz='Michelle Miller')
Quux(xx=171, yy='54596E', zz='Eddie Davis')
Quux(xx=142, yy='2A16D0', zz='Kathleen Lucas')
Quux(xx=140, yy='FDCDD3', zz='Jason Rodriguez')
Quux(xx=121, yy='BDE283', zz='Andrew Pitts')


## Adding generators with dependencies

In [13]:
from tohu.derived_generators import Apply

In [14]:
aa = Integer(1, 9)
bb = Apply(lambda x: x*11, aa)
cc = Apply(lambda x: x*101, bb)

In [19]:
assert bb.arg_gens[0].is_clone_of(aa)
assert cc.arg_gens[0].is_clone_of(bb)

In [20]:
tohu_namespace = TohuNamespaceNEW()
tohu_namespace.add_generator("rr", aa)
tohu_namespace.add_generator("ss", aa)
tohu_namespace.add_generator("tt", bb)
tohu_namespace.add_generator("uu", cc)
tohu_namespace.add_generator("vv", cc)
tohu_namespace.set_tohu_items_class("Quux")

In [21]:
tohu_namespace.generators

{'rr': <Integer (id=b9aaef)>,
 'ss': <Integer (id=7644a3)>,
 'tt': <Apply (id=417a35)>,
 'uu': <Apply (id=5ff032)>,
 'vv': <Apply (id=0ff42e)>}

Note that even though the generator `aa` is added to the namespace twice (first with the name `"rr"` and then with the name `"ss"`), `ss` actually ends up as a _clone_ of `rr` (this is to ensure that the tohu items produced by the namespace contain the correct values).

The following checks that this works as expected.

In [27]:
assert tohu_namespace.generators["ss"].is_clone_of(tohu_namespace.generators["rr"])
assert tohu_namespace.generators["tt"].arg_gens[0].is_clone_of(tohu_namespace.generators["rr"])
assert tohu_namespace.generators["uu"].arg_gens[0].is_clone_of(tohu_namespace.generators["tt"])
assert tohu_namespace.generators["vv"].parent.arg_gens[0].is_clone_of(tohu_namespace.generators["tt"])

tohu_namespace.reset(seed=11111)
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(rr=8, ss=8, tt=88, uu=8888, vv=8888)
Quux(rr=9, ss=9, tt=99, uu=9999, vv=9999)
Quux(rr=6, ss=6, tt=66, uu=6666, vv=6666)
Quux(rr=6, ss=6, tt=66, uu=6666, vv=6666)
Quux(rr=3, ss=3, tt=33, uu=3333, vv=3333)


**TODO:**

Add tests for:
- [X] `.set_tohu_items_class("Quux")`
- [X] `.reset()`
- [X] `.__next__()`
- [X] adding generators with dependencies on previously added generators (check that "rewiring" works correctly)
- [ ] `.extract_loop_runner()`
- [ ] ensure that when `.reset()` is called, only the expected generators are reset (in particular, no clones and no loop variables)

(and any other usages in the notebook `2020-04-08__Prototyping_TohuNamespace_and_LoopRunner_v2.ipynb`)